In [23]:
import pandas as pd
sms_spam = pd.read_csv('SMSSpamCollection.csv', sep='\t',
header=None, names=['Label', 'SMS'])
print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [24]:
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

In [25]:
data_randomized = sms_spam.sample(frac=1, random_state=1)
# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)
# Split into training and test sets
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)
print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [26]:
training_set['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [27]:
test_set['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

In [28]:
training_set.head(3)

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired


In [29]:
training_set['SMS'] = training_set['SMS'].str.replace(
'\W', ' ') # Removes punctuation
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head(3)

C:\Users\SUJAY KHOND\AppData\Local\Temp\ipykernel_13644\3115950900.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set['SMS'] = training_set['SMS'].str.replace(


,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired


In [31]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
vocabulary = list(set(vocabulary))

In [32]:
len(vocabulary)

7783

In [33]:
word_counts_per_sms = {'secret': [2,1,1],
'prize': [2,0,1],
'claim': [1,0,1],
'now': [1,0,1],
'coming': [0,1,0],
'to': [0,1,0],
'my': [0,1,0],
'party': [0,1,0],
'winner': [0,0,1]
}
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,secret,prize,claim,now,coming,to,my,party,winner
0,2,2,1,1,0,0,0,0,0
1,1,0,0,0,1,1,1,1,0
2,1,1,1,1,0,0,0,0,1


In [38]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS'])
                       for unique_word in vocabulary}
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [39]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,argh,prize,xchat,indian,randy,ladies,weirdest,athome,parties,secs,...,lovely,luck,textpod,there,these,dressed,into,commit,threw,salary
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [40]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,argh,prize,xchat,indian,randy,ladies,weirdest,athome,...,lovely,luck,textpod,there,these,dressed,into,commit,threw,salary
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [41]:
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']
# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)
# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()
# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

In [42]:
# N_Vocabulary
n_vocabulary = len(vocabulary)
# Laplace smoothing
alpha = 1

In [45]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}
# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam +alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

In [49]:
import re
def classify(message):
    '''
    message: a string
    '''
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            if word in parameters_ham:
                p_ham_given_message *= parameters_ham[word]
                print('P(Spam|message):', p_spam_given_message)
                print('P(Ham|message):', p_ham_given_message)
                if p_ham_given_message > p_spam_given_message:
                    print('Label: Ham')
                elif p_ham_given_message < p_spam_given_message:
                    print('Label: Spam')
                else:
                    print('Equal proabilities, have a human classify this!')

In [50]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 8.202032944168352e-05
P(Ham|message): 1.3309912303616694e-05
Label: Spam
P(Spam|message): 2.5349076700298304e-07
P(Ham|message): 4.2169208467318346e-08
Label: Spam
P(Spam|message): 1.5116978661650057e-09
P(Ham|message): 3.76163348370419e-10
Label: Spam
P(Spam|message): 1.0396912151398202e-11
P(Ham|message): 5.328305811275852e-12
Label: Spam
P(Spam|message): 3.620567501466313e-15
P(Ham|message): 3.2779488226858514e-16
Label: Spam
P(Spam|message): 3.782423716327494e-18
P(Ham|message): 1.0082893948587669e-20
Label: Spam
P(Spam|message): 9.006162768602878e-20
P(Ham|message): 1.975639324900137e-22
Label: Spam
P(Spam|message): 6.194113600484285e-22
P(Ham|message): 2.7984678840864752e-24
Label: Spam
P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [51]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 5.858594960120251e-06
P(Ham|message): 0.00015971894764340032
Label: Ham
P(Spam|message): 3.06025070828551e-09
P(Ham|message): 4.544448679487705e-07
Label: Ham
P(Spam|message): 2.6642151293131153e-13
P(Ham|message): 2.0967926850912203e-11
Label: Ham
P(Spam|message): 1.1597158095647566e-16
P(Ham|message): 5.933710459193856e-14
Label: Ham
P(Spam|message): 7.067436266010791e-20
P(Ham|message): 1.0859913021286818e-16
Label: Ham
P(Spam|message): 3.9993327583746263e-22
P(Ham|message): 1.377949591289084e-18
Label: Ham
P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


In [53]:
def classify_test_set(message):
    '''
    message: a string
    '''
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            if word in parameters_ham:
                p_ham_given_message *= parameters_ham[word]
                if p_ham_given_message > p_spam_given_message:
                    return 'ham'
                elif p_spam_given_message > p_ham_given_message:
                    return 'spam'
                else:
                    return 'needs human classification'

In [54]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,ham
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [56]:
correct = 0
total = test_set.shape[0]
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        print('Correct:', correct)
        print('Incorrect:', total - correct)
        print('Accuracy:', correct/total)

Correct: 1
Incorrect: 1113
Accuracy: 0.0008976660682226212
Correct: 2
Incorrect: 1112
Accuracy: 0.0017953321364452424
Correct: 3
Incorrect: 1111
Accuracy: 0.0026929982046678637
Correct: 4
Incorrect: 1110
Accuracy: 0.003590664272890485
Correct: 5
Incorrect: 1109
Accuracy: 0.004488330341113106
Correct: 6
Incorrect: 1108
Accuracy: 0.005385996409335727
Correct: 7
Incorrect: 1107
Accuracy: 0.0062836624775583485
Correct: 8
Incorrect: 1106
Accuracy: 0.00718132854578097
Correct: 9
Incorrect: 1105
Accuracy: 0.00807899461400359
Correct: 10
Incorrect: 1104
Accuracy: 0.008976660682226212
Correct: 11
Incorrect: 1103
Accuracy: 0.009874326750448833
Correct: 12
Incorrect: 1102
Accuracy: 0.010771992818671455
Correct: 13
Incorrect: 1101
Accuracy: 0.011669658886894075
Correct: 14
Incorrect: 1100
Accuracy: 0.012567324955116697
Correct: 15
Incorrect: 1099
Accuracy: 0.013464991023339317
Correct: 16
Incorrect: 1098
Accuracy: 0.01436265709156194
Correct: 17
Incorrect: 1097
Accuracy: 0.01526032315978456
Correc